In [140]:
import numpy as np
import subprocess
import os
from Bio import SeqIO
from joblib import Parallel, delayed
import multiprocessing as mp
from datetime import datetime

In [133]:
# Functions

def oneHot(residue):
    mapping = dict(zip("ACDEFGHIKLMNPQRSTVWY", range(20)))
    if residue in "ACDEFGHIKLMNPQRSTVWY":
        return np.eye(20)[mapping[residue]]
    else:
        return np.zeros(20)
    
    
def reverseOneHot(encoding):
    mapping = dict(zip(range(20),"ACDEFGHIKLMNPQRSTVWY"))
    seq=''
    for i in range(len(encoding)):
        if np.max(encoding[i])>0:
            seq+=mapping[np.argmax(encoding[i])]
    return seq

def foldx(model):
    os.chdir('/home/ida/foldx')

    # RepairPDB
    repair_command = "./foldx --command=RepairPDB --pdb={} --ionStrength=0.05 --pH=7 --water=CRYSTAL --vdwDesign=2 --out-pdb=1 --pdbHydrogens=false".format(model)
    subprocess.run(repair_command.split(),
                        stdout=subprocess.PIPE, universal_newlines=True)
    
    # AnalyseComplex
    analyse_command = "./foldx --command=AnalyseComplex --pdb=" + model.replace(".pdb", "_Repair.pdb")

In [134]:
# Import input file

fastas = ["/home/ida/master-thesis/5nmf.fsa", "/home/ida/master-thesis/5nmf_mut.fsa"]
infilename = "/home/ida/master-thesis/5nmf.fsa"


In [135]:
# Model complex with TCRpMHCmodels

path_tcrpmhcmodels = "/home/ida/TCRpMHCmodels/"

path_fasta = "/home/ida/master-thesis/5nmf.fsa"

#subprocess.run(["tcrpmhc_models", "-t", path_fasta])

# Move output pdb to foldx folder

In [136]:
# Calculate energy Rosetta

# Relax
relax_command = "relax.default.linuxgccrelease -s model_TCR-pMHC.pdb -nstruct 5 -relax:default_repeats 5 -out:path:pdb /home/people/idamei/relax_output -out:path:score /home/people/idamei/expeced_output"
# Relax cartesian
relax_command = "relax.default.linuxgccrelease -s model_TCR-pMHC.pdb -nstruct 5 -relax:default_repeats 5 -out:path:pdb /home/people/idamei/relax_output -out:path:score /home/people/idamei/expeced_output -relax:dualspace true"

# Score per residue
per_res_command = "per_residue_energies.linuxgccrelease -in:file:s /home/people/idamei/relax_output/model_TCR-pMHC_0001.pdb"

# Score
score_command = "score_jd2.linuxgccrelease -in:file:s /home/people/idamei/relax_output/model_TCR-pMHC_0001.pdb"

In [138]:
# Calculate energy with FoldX
os.chdir('/home/ida/foldx')

models = ["5men_model_TCR-pMHC.pdb", "5nmf_model_TCR-pMHC.pdb", "3tkf_model_TCR-pMHC.pdb", "5isz_model_TCR-pMHC.pdb"]

#jobs = list()
#for model in models:
#    jobs.append("./foldx --command=RepairPDB --pdb={} --ionStrength=0.05 --pH=7 --water=CRYSTAL --vdwDesign=2 --out-pdb=1 --pdbHydrogens=false".format(model))

#Parallel(n_jobs=2)(delayed(foldx)(x) for x in models)

pool = mp.Pool(4)

pool.map(foldx, [model for model in models])

pool.close()

start 5nmf_model_TCR-pMHC.pdb
start 3tkf_model_TCR-pMHC.pdb
start 5isz_model_TCR-pMHC.pdb

start 5men_model_TCR-pMHC.pdb



finish 5men_model_TCR-pMHC.pdbtime: 0:14:14.775441

finish 5nmf_model_TCR-pMHC.pdbtime: 0:14:44.710414

finish 3tkf_model_TCR-pMHC.pdbtime: 0:15:29.137440

finish 5isz_model_TCR-pMHC.pdbtime: 0:16:12.590355

{'MP': '-4.57047', 'MA': '-0.925445', 'MB': '-2.24605', 'PA': '-0.493195', 'PB': '0', 'AB': '-1.554'}


In [66]:
# Create output

for AA in peptide_seq:
    print(oneHot(AA))
    
    
foldx_output_path = "/home/ida/foldx/Interaction_model_TCR-pMHC_Repair_AC.fxout"
foldx_output = open(foldx_output_path, "r")
foldx_interaction_energies = dict()
for line in foldx_output:
    if line.startswith("./"):
        splitted_line = line.split("\t")
        group1 = splitted_line[1]
        group2 = splitted_line[2]
        interaction_energy = splitted_line[6]
        foldx_interaction_energies[group1+group2] = interaction_energy
        
print(foldx_interaction_energies)

NameError: name 'peptide_seq' is not defined

In [190]:
def create_output(pdb_path):

    pdb_sequence = SeqIO.parse(open(pdb_path), 'pdb-atom')

    sequences = []
    chains = []
    nrows = 0
    for chain in pdb_sequence:
        sequences.append(str(chain.seq))
        nrows += len(str(chain.seq))
        chains.append(chain.id.replace('????:', ''))
    
    print(sequences)
    print(chains)
    print(nrows)
    
    output_array = np.empty(shape=(nrows,22))
    k1 = 0
    for chain in sequences:
        for aminoacid in chain:
            print(oneHot(aminoacid))
            output_array[k1,:] = oneHot(aminoacid)
    #np.asarray(oneHot)

In [191]:
create_output("/home/ida/foldx/5men_model_TCR-pMHC.pdb")

/home/ida/anaconda3/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3243
  warnings.warn(
/home/ida/anaconda3/lib/python3.8/site-packages/Bio/SeqIO/PdbIO.py:303: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(


['QVEQSPPDLILQEGANSTLRCNFSDSXXXXXXXXVNNLQWFHQNXPWGQLINLFYIPXXXXXXSGTKQXNXGRLSATTXVXATERYSLLYISSSQTTDSGVYFCAVDSATSXXXXXXXXGTYKYIFGTGTRLKV', 'AGVTQTPKFQVLKTGQSMTLQCAQDMXXXXXXNHEYMSWYRQDPGMGLRLIHYSVGAGITDXXXXXXXQGEVPNGYNVSRSTTEDFPLRLLSAAPSQTSVYFCASSYQXXXXXXXXXXXXXXXGTEAFFGQGTRLTVV', 'GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGXPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQXLRAYLEGTCVEWLRRYLENGKETL', 'ILAKFLHWL']
['A', 'B', 'M', 'P']
462
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0